In [2]:
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder.appName('Итоговое_задание').getOrCreate()

In [44]:
df = spark.read.csv('web_server_logs.csv', header=True)

In [46]:
df.show(1)

+-------------+-------------------+------+--------+-------------+-------------+
|           ip|          timestamp|method|     url|response_code|response_size|
+-------------+-------------------+------+--------+-------------+-------------+
|62.139.73.167|2025-02-04T23:56:12|  POST|category|          404|         8385|
+-------------+-------------------+------+--------+-------------+-------------+
only showing top 1 row



In [48]:
df.printSchema()

root
 |-- ip: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: string (nullable = true)
 |-- response_size: string (nullable = true)



In [52]:
from pyspark.sql.functions import col, to_timestamp
df = df.withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss"))

In [54]:
df.printSchema()

root
 |-- ip: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- method: string (nullable = true)
 |-- url: string (nullable = true)
 |-- response_code: string (nullable = true)
 |-- response_size: string (nullable = true)



**1. Сгруппируйте данные по IP и посчитайте количество запросов для каждого IP, выводим 10 самых активных IP. Формат вывода, как на скрине ниже.**

In [66]:
from pyspark.sql.functions import count
df.groupBy("ip").count().orderBy(col("count").desc()).limit(10).show()

+---------------+-----+
|             ip|count|
+---------------+-----+
|  181.98.103.51|    2|
|  54.33.128.250|    1|
| 14.162.102.216|    1|
| 43.159.224.211|    1|
| 120.212.152.60|    1|
|108.138.189.237|    1|
| 120.245.53.106|    1|
|  206.61.11.202|    1|
|   156.117.46.5|    1|
|    39.35.0.189|    1|
+---------------+-----+



**2. Сгруппируйте данные по HTTP-методу и посчитайте количество запросов для каждого метода.**

In [106]:
df.groupBy("method").count().withColumnRenamed("count", "method_count").show()

+------+------------+
|method|method_count|
+------+------------+
|  POST|       25106|
|DELETE|       24852|
|   PUT|       25066|
|   GET|       24976|
+------+------------+



**3. Профильтруйте и посчитайте количество запросов с кодом ответа 404.**

In [82]:
df.where(col('response_code') == 404).groupBy('response_code').count().show()

+-------------+-----+
|response_code|count|
+-------------+-----+
|          404|25250|
+-------------+-----+



**4. Сгруппируйте данные по дате и просуммируйте размер ответов, сортируйте по дате.**

In [100]:
from pyspark.sql.functions import sum, to_date
df = df.withColumn("response_size", col("response_size").cast("int"))
df = df.withColumn('date', to_date(col('timestamp')))
df.groupBy('date').sum('response_size').orderBy('date').show()

+----------+------------------+
|      date|sum(response_size)|
+----------+------------------+
|2025-01-01|           6277068|
|2025-01-02|           6271167|
|2025-01-03|           6156048|
|2025-01-04|           6413230|
|2025-01-05|           5983700|
|2025-01-06|           6258091|
|2025-01-07|           6140001|
|2025-01-08|           6091521|
|2025-01-09|           6205761|
|2025-01-10|           6295853|
|2025-01-11|           6106426|
|2025-01-12|           6364797|
|2025-01-13|           6138723|
|2025-01-14|           6233212|
|2025-01-15|           6104700|
|2025-01-16|           6085768|
|2025-01-17|           5836398|
|2025-01-18|           6513908|
|2025-01-19|           6012376|
|2025-01-20|           6208759|
+----------+------------------+
only showing top 20 rows

